# Postprocess the evaluation results

```shell
python evaluate_factual_subgraphs.py 
  -d ../resources/datasets/processed/wikipedia 
  --cuda 
  --bipartite 
  --model ../resources/models/wikipedia/wikipedia-wikipedia.pth 
  --type TGAT
  --results ../resources/results/wikipedia/tgnnexplainer/results_wikipedia_tgnnexplainer.parquet
```



```shell
python evaluate_fidelity_minus.py
  -d ../resources/datasets/processed/wikipedia
  --cuda
  --bipartite
  --model ../resources/models/wikipedia/wikipedia-wikipedia.pth
  --type TGAT
  --results ../results/wikipedia/greedy
  --all_files_in_dir
```




In [ ]:
# Find and add `notebooks/src` to sys.path, no matter where the notebook lives.
from pathlib import Path
import sys, importlib
import os
import pandas as pd
import numpy as np
from os import listdir

def _add_notebooks_src_to_path():
    here = Path.cwd().resolve()
    for p in [here, *here.parents]:
        candidate = p / "notebooks" / "src"
        if candidate.is_dir():
            if str(candidate) not in sys.path:
                sys.path.insert(0, str(candidate))
            return candidate
    raise FileNotFoundError("Could not find 'notebooks/src' from current working directory.")

print("Using helpers from:", _add_notebooks_src_to_path())

from constants import (
    REPO_ROOT, PKG_DIR, RESOURCES_DIR, PROCESSED_DATA_DIR, MODELS_ROOT, RESULTS_ROOT, TGN_SUBMODULE_ROOT, ensure_repo_importable, get_last_checkpoint
)
ensure_repo_importable()
from device import pick_device

for p in (str(TGN_SUBMODULE_ROOT), str(REPO_ROOT), str(PKG_DIR)):
    if p not in sys.path:
        sys.path.insert(0, p)

# 2) If your notebook already imported `utils`, remove it to avoid collision
if "utils" in sys.modules:
    del sys.modules["utils"]

importlib.invalidate_caches()

# 4) (Optional) sanity check that TGN's local packages resolve
import importlib.util as iu
print("utils.utils   ->", iu.find_spec("utils.utils"))
print("modules.memory->", iu.find_spec("modules.memory"))

# 5) Now this import should work without the previous error
from time_to_explain.explainer.evaluate_factual_subgraphs import (
    evaluate_explainer_results
)
from time_to_explain.metrics.evaluate_fidelity_minus import (
from time_to_explain.core.factories import build_dataset, build_model
    add_prediction_for_dataframe, save_results_to_file
)

print("REPO_ROOT        :", REPO_ROOT)
print("PKG_DIR          :", PKG_DIR)
print("RESOURCES_DIR    :", RESOURCES_DIR)
print("PROCESSED_DATA_DIR:", PROCESSED_DATA_DIR)
print("MODELS_ROOT      :", MODELS_ROOT)

Using helpers from: /Users/juliawenkmann/Documents/CodingProjects/master_thesis/time_to_explain/notebooks/src
utils.utils   -> ModuleSpec(name='utils.utils', loader=<_frozen_importlib_external.SourceFileLoader object at 0x11352f610>, origin='/Users/juliawenkmann/Documents/CodingProjects/master_thesis/time_to_explain/submodules/models/tgn/utils/utils.py')
modules.memory-> ModuleSpec(name='modules.memory', loader=<_frozen_importlib_external.SourceFileLoader object at 0x11355a050>, origin='/Users/juliawenkmann/Documents/CodingProjects/master_thesis/time_to_explain/submodules/models/tgn/modules/memory.py')


ImportError: cannot import name 'add_wrapper_model_arguments' from 'time_to_explain.data.common' (/Users/juliawenkmann/Documents/CodingProjects/master_thesis/time_to_explain/time_to_explain/data/common.py)

## Setting:

In [ ]:
MODEL_TYPE = "TGAT"
DATASET_NAME = "wikipedia"
DIRECTED = False
EXPLAINER = "cody"
SELECTION_NAME = "all"
BIPARTITE = True


MODEL_PATH = MODELS_ROOT / DATASET_NAME
RESULTS_PATH = RESULTS_ROOT / DATASET_NAME /EXPLAINER 
if not os.path.exists(RESULTS_PATH):
    os.makedirs(RESULTS_PATH)
RESULT_FILE = RESULTS_PATH / f"results_{DATASET_NAME}_{EXPLAINER}.parquet"

ALL_FILES_IN_DIR = True


DEVICE = pick_device("auto")
print(DEVICE)

mps


To finish the evaluation and enrich the evaluation datasets with further information we need 2 more steps:

## Enrich Factual Explanations:

The fist step is to add the prediction that is achieved when removing the events in the explanation produced by 
T-GNNExplainer from the input data (to evaluate fidelity+/whether the explanation is counterfactual). To add this 
information, use the [evaluate_factual_subgraphs.py](/scripts/evaluate_factual_subgraphs.py) script. This adds the information to the results for correct predictions only for the wikipedia dataset.

In [ ]:
dataset_cfg = {
    "builder": "processed",
    "path": str(PROCESSED_DATA_DIR / DATASET_NAME),
    "directed": DIRECTED,
    "bipartite": BIPARTITE,
}
dataset_bundle = build_dataset(dataset_cfg)
dataset = dataset_bundle.obj

model_cfg = {
    "builder": MODEL_TYPE.lower(),
    "device": DEVICE.type if hasattr(DEVICE, "type") else str(DEVICE),
}
last_checkpoint = get_last_checkpoint(MODEL_PATH / 'checkpoints', MODEL_TYPE, DATASET_NAME)
if last_checkpoint:
    model_cfg["checkpoint"] = str(last_checkpoint)
model = build_model(model_cfg, dataset)
if hasattr(model, "set_evaluation_mode"):
    model.set_evaluation_mode(True)

evaluate_explainer_results(
    results_path=RESULT_FILE,
    tgn_wrapper=model,
    show_progress=True,
    drop_first_csv_col=True,
    save_outputs=True,
    save_csv=True,
    save_parquet=True,
)


## Add Fidelity- information:

The second step is to analyze how much the events in the explanation influence the prediction. For this, use the 
[evaluate_fidelity_minus.py](/scripts/evaluate_fidelity_minus.py). This adds the fidelity- information to all the results saved in the _results/wikipedia/greedy_ directory. Rerun the 
command for each of the explainers and each of the datasets to add the information to all results.


In [ ]:
if hasattr(model, "set_evaluation_mode"):
    model.set_evaluation_mode(True)

if not os.path.exists(RESULT_FILE):
    raise FileNotFoundError('Failed to locate the file containing the results')

dataframes = []
if ALL_FILES_IN_DIR:
    results_files = [f for f in listdir(RESULTS_PATH) if os.path.isfile(os.path.join(RESULTS_PATH, f)) and f.endswith('parquet')]
    for results_file in results_files:
        filepath = f'{RESULTS_PATH}/{results_file}'
        results = pd.read_parquet(filepath)
        dataframes.append((results, filepath.rstrip('parquet')))
else:
    if RESULT_FILE.endswith('parquet'):
        results = pd.read_parquet(RESULT_FILE)
        dataframes.append((results, RESULT_FILE.rstrip('parquet')))
    else:
        raise RuntimeError('Cannot read results. Only parquet files supported.')

for results, filename in dataframes:
    results = add_prediction_for_dataframe(results, model, dataset)
    save_results_to_file(results, filename)
